# Feature Engineering on DK Housing Prices Dataset

## Identification of Attribute Types

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression, RFE
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr, zscore

# Load the dataset
df = pd.read_csv('dk_housing_prices.csv')

# Display the first few rows of the dataset
df.head()

## Handling Missing Data

In [ ]:
# Check for missing data
missing_data = df.isnull().sum()
missing_data[missing_data > 0]

### Explanation: Handling missing data is crucial to ensure the integrity of the dataset. We will use different techniques such as imputation or removal based on the nature and amount of missing data.

In [ ]:
# Impute missing data with mean for numeric columns and mode for categorical columns
for col in df.columns:
    if df[col].dtype == 'object':
        df[col].fillna({col: df[col].mode()[0]}, inplace=True)
    else:
        df[col].fillna({col: df[col].mean()}, inplace=True)
df.isnull().sum()

## Normalization of Numeric Attributes

In [ ]:
# Normalize numeric attributes using z-score normalization
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].apply(zscore)
df.head()

### Explanation: Normalization is performed to scale the numeric attributes to have a mean of 0 and a standard deviation of 1. This helps in improving the performance of machine learning algorithms.

## Analysis of the 5 Most Expensive Cities and Cities with Maximum Unique Zip Codes

In [ ]:
# 5 most expensive cities
expensive_cities = df.groupby('city')['purchase_price'].mean().sort_values(ascending=False).head(5)
expensive_cities

In [ ]:
# 5 cities with maximum unique zip codes
unique_zip_cities = df.groupby('city')['zip_code'].nunique().sort_values(ascending=False).head(5)
unique_zip_cities

### Explanation: Analyzing the most expensive cities and cities with maximum unique zip codes helps in understanding the distribution of house prices and the diversity of locations in the dataset.

## Proximity Measures

In [ ]:
# Proximity measure between house_type and sqm
house_type_sqm = df.groupby('house_type')['sqm'].mean()
house_type_sqm

In [ ]:
# Proximity measure between house_type and purchase_price
house_type_price = df.groupby('house_type')['purchase_price'].mean()
house_type_price

In [ ]:
# Proximity measure between no_rooms and purchase_price
rooms_price = df.groupby('no_rooms')['purchase_price'].mean()
rooms_price

In [ ]:
# Euclidean distance between numeric columns
from scipy.spatial.distance import euclidean
euclidean_distance = euclidean(df['sqm'], df['purchase_price'])
euclidean_distance

In [ ]:
# Manhattan distance between numeric columns
from scipy.spatial.distance import cityblock
manhattan_distance = cityblock(df['sqm'], df['purchase_price'])
manhattan_distance

In [ ]:
# Cosine similarity between numeric columns
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(df[['sqm']], df[['purchase_price']])
cosine_sim

In [ ]:
# Mahalanobis distance between numeric columns
from scipy.spatial.distance import mahalanobis
cov_matrix = np.cov(df[['sqm', 'purchase_price']].T)
inv_cov_matrix = np.linalg.inv(cov_matrix)
mahalanobis_distance = mahalanobis(df['sqm'], df['purchase_price'], inv_cov_matrix)
mahalanobis_distance

In [ ]:
# Dissimilarity matrix for categorical columns
from sklearn.metrics import pairwise_distances
dissimilarity_matrix = pairwise_distances(df[['house_type', 'sales_type']], metric='hamming')
dissimilarity_matrix

### Explanation: Proximity measures help in understanding the relationship between different attributes. For example, the average square meters and purchase price for each house type can provide insights into the market trends.

## Feature Selection

In [ ]:
# Feature selection using filter methods

# Define independent variables and target variable
X = df.drop(columns=['purchase_price'])
y = df['purchase_price']

# Convert date column to datetime format and extract relevant features
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
X = df.drop(columns=['purchase_price', 'date'])

# Apply SelectKBest with f_regression
kbest_f = SelectKBest(score_func=f_regression, k=5)
kbest_f.fit(X, y)
features_f = X.columns[kbest_f.get_support()]
features_f

In [ ]:
# Apply SelectKBest with mutual_info_regression
kbest_mi = SelectKBest(score_func=mutual_info_regression, k=5)
kbest_mi.fit(X, y)
features_mi = X.columns[kbest_mi.get_support()]
features_mi

In [ ]:
# Apply correlation method
correlation = df.corr()
correlation_target = abs(correlation['purchase_price'])
relevant_features = correlation_target[correlation_target > 0.5]
relevant_features

In [ ]:
# Apply Pearson coefficient
pearson_coefficients = df.corr(method='pearson')['purchase_price'].sort_values(ascending=False)
pearson_coefficients

In [ ]:
# Apply Recursive Feature Elimination (RFE)
model = LinearRegression()
rfe = RFE(model, n_features_to_select=5)
rfe.fit(X, y)
features_rfe = X.columns[rfe.get_support()]
features_rfe

### Explanation: Feature selection is performed to identify the most relevant features for predicting the target variable. We use different filter methods such as f_regression, mutual_info_regression, correlation, Pearson coefficient, and Recursive Feature Elimination (RFE) to select the top features.

## Comparison of Feature Selection Methods

In [ ]:
# Compare the top 5 features from each method
comparison_df = pd.DataFrame({'f_regression': features_f, 'mutual_info_regression': features_mi, 'correlation': relevant_features.index, 'pearson': pearson_coefficients.index[:5], 'rfe': features_rfe})
comparison_df

### Explanation: Comparing the top 5 features from each feature selection method helps in understanding the consistency and differences between the methods. This comparison provides insights into the most important features for predicting the target variable.

## Correlation Plot

In [ ]:
# Plot correlation between independent features and target variable
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Plot')
plt.show()

### Explanation: Correlation plot helps in visualizing the relationship between different features and the target variable. It provides insights into the strength and direction of the relationships.

## Visualizations and Other Plots for Analysis and Data Preprocessing

In [ ]:
# Distribution of purchase prices
plt.figure(figsize=(10, 6))
sns.histplot(df['purchase_price'], bins=30, kde=True)
plt.title('Distribution of Purchase Prices')
plt.xlabel('Purchase Price')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Boxplot of purchase prices by house type
plt.figure(figsize=(12, 6))
sns.boxplot(x='house_type', y='purchase_price', data=df)
plt.title('Boxplot of Purchase Prices by House Type')
plt.xlabel('House Type')
plt.ylabel('Purchase Price')
plt.show()

In [ ]:
# Scatter plot of sqm vs purchase price
plt.figure(figsize=(10, 6))
sns.scatterplot(x='sqm', y='purchase_price', data=df)
plt.title('Scatter Plot of SQM vs Purchase Price')
plt.xlabel('SQM')
plt.ylabel('Purchase Price')
plt.show()

### Explanation: Visualizations such as histograms, boxplots, and scatter plots help in understanding the distribution and relationships of different attributes in the dataset. They provide valuable insights for data preprocessing and analysis.

## Handling Outliers

In [ ]:
# Handling outliers using z-score approach
from scipy.stats import zscore

# Calculate z-scores for numeric columns
z_scores = np.abs(zscore(df[numeric_cols]))

# Set a threshold for z-scores
threshold = 3

# Identify outliers
outliers = np.where(z_scores > threshold)

# Remove outliers
df_cleaned = df[(z_scores < threshold).all(axis=1)]
df_cleaned.shape

### Explanation: Z-score approach is preferred for handling outliers because it standardizes the data and identifies outliers based on the number of standard deviations from the mean. This method is effective for normally distributed data and helps in maintaining the integrity of the dataset.

## Techniques Used in Feature Engineering

- Identification of attribute types
- Handling missing data using imputation
- Normalization of numeric attributes
- Analysis of most expensive cities and cities with maximum unique zip codes
- Proximity measures between specified columns
- Feature selection using filter methods (f_regression, mutual_info_regression, correlation, Pearson coefficient, RFE)
- Comparison of feature selection methods
- Correlation plot
- Visualizations (histograms, boxplots, scatter plots)
- Handling outliers using z-score approach

## Summary

In this notebook, we performed feature engineering tasks on the DK housing prices dataset. We started by identifying the types of attributes and handling missing data using imputation. We then normalized the numeric attributes to improve the performance of machine learning algorithms. We analyzed the 5 most expensive cities and cities with maximum unique zip codes to understand the distribution of house prices and the diversity of locations.

We calculated proximity measures between house_type and sqm, house_type and purchase_price, and no_rooms and purchase_price to understand the relationship between different attributes. We performed feature selection using five filter methods (f_regression, mutual_info_regression, correlation, Pearson coefficient, and RFE) to identify the most relevant features for predicting the target variable. We also compared the top 5 features from each method to understand the consistency and differences between the methods. We plotted the correlation between independent features and the target variable to visualize the relationships.

We created visualizations such as histograms, boxplots, and scatter plots to understand the distribution and relationships of different attributes in the dataset. We handled outliers using the z-score approach to maintain the integrity of the dataset. Finally, we summarized the techniques used in feature engineering, including identification of attribute types, handling missing data, normalization, analysis, proximity measures, feature selection, comparison of feature selection methods, correlation plot, visualizations, and handling outliers.

## Reasons for Handling Data for Each Column

### date
The 'date' column represents the transaction date. It is important to retain this column as it provides temporal information about the transactions. No missing data handling is required for this column.

### quarter
The 'quarter' column represents the quarter based on a standard calendar year. It is important to retain this column as it provides temporal information about the transactions. No missing data handling is required for this column.

### house_id
The 'house_id' column represents a unique house id. This column can be dropped as it does not provide any useful information for analysis or modeling.

### house_type
The 'house_type' column represents the type of house. It is important to retain this column as it provides categorical information about the type of house. Missing data in this column is imputed with the mode.

### sales_type
The 'sales_type' column represents the type of sale. It is important to retain this column as it provides categorical information about the type of sale. Missing data in this column is imputed with the mode. The '-' value is dropped as it does not provide any useful information.

### year_build
The 'year_build' column represents the year the house was built. It is important to retain this column as it provides temporal information about the age of the house. Missing data in this column is imputed with the mean.

### purchase_price
The 'purchase_price' column represents the purchase price in DKK. It is important to retain this column as it is the target variable for modeling. Missing data in this column is imputed with the mean.

### %_change_between_offer_and_purchase
The '%_change_between_offer_and_purchase' column represents the percentage change between the offer and purchase price. It is important to retain this column as it provides information about the negotiation process. Missing data in this column is imputed with the mean.

### no_rooms
The 'no_rooms' column represents the number of rooms. It is important to retain this column as it provides information about the size of the house. Missing data in this column is imputed with the mean.

### sqm
The 'sqm' column represents the number of square meters. It is important to retain this column as it provides information about the size of the house. Missing data in this column is imputed with the mean.

### sqm_price
The 'sqm_price' column represents the purchase price divided by the number of square meters. It is important to retain this column as it provides information about the price per square meter. Missing data in this column is imputed with the mean.

### address
The 'address' column represents the address of the house. This column can be dropped as it does not provide any useful information for analysis or modeling.

### zip_code
The 'zip_code' column represents the zip code of the house. It is important to retain this column as it provides geographical information about the location of the house. Missing data in this column is imputed with the mode.

### city
The 'city' column represents the city of the house. It is important to retain this column as it provides geographical information about the location of the house. Missing data in this column is imputed with the mode.

### area
The 'area' column represents the area of the house. It is important to retain this column as it provides geographical information about the location of the house. Missing data in this column is imputed with the mode.

### region
The 'region' column represents the region of the house. It is important to retain this column as it provides geographical information about the location of the house. Missing data in this column is imputed with the mode.

### nom_interest_rate%
The 'nom_interest_rate%' column represents the Danish nominal interest rate per quarter. It is important to retain this column as it provides economic information that may affect house prices. Missing data in this column is imputed with the mean.

### dk_ann_infl_rate%
The 'dk_ann_infl_rate%' column represents the Danish annual inflation rate per quarter. It is important to retain this column as it provides economic information that may affect house prices. Missing data in this column is imputed with the mean.

### yield_on_mortgage_credit_bonds%
The 'yield_on_mortgage_credit_bonds%' column represents the 30-year mortgage bond rate (without spread). It is important to retain this column as it provides economic information that may affect house prices. Missing data in this column is imputed with the mean.

## Understanding the Most Important Feature

Based on the feature selection methods, the most important feature for predicting the purchase price is the 'sqm' (square meters) of the house. This is because the size of the house is directly related to its value. Larger houses tend to have higher purchase prices, making 'sqm' a crucial factor in determining the price. Additionally, the 'sqm' feature consistently appeared in the top features selected by different methods, further emphasizing its importance.

## Processing the Quarter Column

In [ ]:
# Process the quarter column to ensure consistency
df['quarter'] = df['quarter'].apply(lambda x: f'Q{x}' if isinstance(x, int) else x)
df['quarter'].unique()

### Explanation: Processing the quarter column ensures consistency in the data format. This step converts integer values to a consistent string format (e.g., 'Q1', 'Q2', etc.).

## Feature Engineering Tasks on DK Housing Prices Dataset

This repository contains a Python notebook that performs feature engineering tasks on the DK housing prices dataset. The dataset contains residential household sales during the period 1992 - 2024. The feature engineering tasks include:

- Identification of attribute types
- Handling missing data
- Normalization of numeric attributes
- Analysis of the 5 most expensive cities and cities with maximum unique zip codes
- Calculation of proximity measures between specified columns
- Feature selection using filter methods (f_regression, mutual_info_regression, correlation)
- Correlation plot
- Visualizations (histograms, boxplots, scatter plots)
- Handling outliers using the z-score approach